In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import zipfile
zip_path = "/content/drive/My Drive/Projects/Image Classification/image-detect.zip"
zip_ref = zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall("/content/")
zip_ref.close()
print ("ZIP Data Extracted Successfully")

In [ ]:
# Imports for Deep Learning
import tensorflow as tf
import tensorflow_addons as tfa
import os
import numpy as np
import pandas as pd
import tqdm
from tensorflow import cast
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

# Paths and Parameters
train_path = "train/"
val_path = "val/images/"
val_labels_path = "val/val_annotations.txt"
test_path = "test/"

# Transfer Learning Model
from tensorflow.keras.applications import InceptionV3
transfer_model = InceptionV3

# Baseline Model (128 x 128)

In [ ]:
# Hyperparameters
batch_size = 100
num_classes = 200
no_epochs = 10
target_size = (128, 128)
input_shape = (128, 128, 3)

In [ ]:
# Create hash map for images and corresponding labels for Training data
train_data = []
for folder in sorted(os.listdir(train_path)):
    for file in sorted(os.listdir(train_path+folder+'/images')):
        train_data.append((folder, file))
training_df = pd.DataFrame(train_data, columns=['label', 'id'])
training_df['filename'] = train_path+training_df['label']+'/images/'+ training_df['id']

# Create hash map for images and corresponding labels for Validation data
validation_df = pd.read_csv(val_labels_path,sep="\t",header=None)
validation_df.columns=["id","label","X_min","X_max","Y_min","Y_max"]
validation_df['filename'] = val_path + validation_df['id']

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = image_generator.flow_from_dataframe(
    dataframe=training_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",   
    shuffle=True)

val_generator = image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True)    

test_generator = image_generator.flow_from_directory(
    directory=test_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

In [ ]:
base_model = transfer_model(weights=None,
                            include_top=False,
                            input_shape = input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(200,activation='softmax')(x)
model = Model(base_model.input, predictions)

In [ ]:
model.compile(optimizer="RMSprop", loss='categorical_crossentropy', metrics=["accuracy"])  
model.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath="model_base128.hdf5", verbose=1, save_best_only=True, monitor="val_accuracy")

In [ ]:
history = model.fit(train_generator, 
                    epochs=no_epochs, 
                    callbacks=[checkpointer], 
                    validation_data=val_generator, 
                    verbose = 1,
                    workers = 20)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('1.jpg')

In [ ]:
fig2 = plt.figure(figsize=(5,5))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.show()
fig2.savefig('2.jpg')

# Baseline Model with Data Augmentation (128 x 128)

In [ ]:
# Hyperparameters
batch_size = 100
num_classes = 200
no_epochs = 10
target_size = (128, 128)
input_shape = (128, 128, 3)

In [ ]:
# Create hash map for images and corresponding labels for Training data
train_data = []
for folder in sorted(os.listdir(train_path)):
    for file in sorted(os.listdir(train_path+folder+'/images')):
        train_data.append((folder, file))
training_df = pd.DataFrame(train_data, columns=['label', 'id'])
training_df['filename'] = train_path+training_df['label']+'/images/'+ training_df['id']

# Create hash map for images and corresponding labels for Validation data
validation_df = pd.read_csv(val_labels_path,sep="\t",header=None)
validation_df.columns=["id","label","X_min","X_max","Y_min","Y_max"]
validation_df['filename'] = val_path + validation_df['id']

image_generator = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = image_generator.flow_from_dataframe(
    dataframe=training_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",   
    shuffle=True)

val_generator = image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True)    

test_generator = image_generator.flow_from_directory(
    directory=test_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

data_augmentation = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomRotation(0.3),
        layers.experimental.preprocessing.Normalization()
        layers.experimental.preprocessing.RandomFlip(),
        layers.experimental.preprocessing.RandomContrast(0.3),
        layers.experimental.preprocessing.RandomTranslation(0.2, 0.2, fill_mode='wrap', interpolation='bilinear')
    ]
)

In [ ]:
inputs = Input(input_shape)
x = data_augmentation(inputs)
base_model = transfer_model(weights=None,
                            include_top=False,
                            input_shape = input_shape)
x = base_model(x)
x = GlobalAveragePooling2D()(x)
predictions = Dense(200,activation='softmax')(x)
model = Model(inputs, predictions)

In [ ]:
model.compile(optimizer="RMSprop", loss='categorical_crossentropy', metrics=["accuracy"])  
model.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath="model_aug128.hdf5", verbose=1, save_best_only=True, monitor="val_accuracy")

In [ ]:
history = model.fit(train_generator, 
                    epochs=no_epochs, 
                    callbacks=[checkpointer], 
                    validation_data=val_generator, 
                    verbose = 1,
                    workers = 20)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('3.jpg')

In [ ]:
fig2 = plt.figure(figsize=(5,5))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
fig2.savefig('4.jpg')

# Baseline Model with Pretrained Weights (128 x 128)

In [ ]:
# Hyperparameters
batch_size = 100
num_classes = 200
no_epochs = 10
target_size = (128, 128)
input_shape = (128, 128, 3)

In [ ]:
# Create hash map for images and corresponding labels for Training data
train_data = []
for folder in sorted(os.listdir(train_path)):
    for file in sorted(os.listdir(train_path+folder+'/images')):
        train_data.append((folder, file))
training_df = pd.DataFrame(train_data, columns=['label', 'id'])
training_df['filename'] = train_path+training_df['label']+'/images/'+ training_df['id']

# Create hash map for images and corresponding labels for Validation data
validation_df = pd.read_csv(val_labels_path,sep="\t",header=None)
validation_df.columns=["id","label","X_min","X_max","Y_min","Y_max"]
validation_df['filename'] = val_path + validation_df['id']

image_generator = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = image_generator.flow_from_dataframe(
    dataframe=training_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",   
    shuffle=True)

val_generator = image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True)    

test_generator = image_generator.flow_from_directory(
    directory=test_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

In [ ]:
base_model = transfer_model(weights="imagenet",
                            include_top=False,
                            input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(200,activation='softmax')(x)
model_pre = Model(base_model.input, predictions)
"""for layer in base_model.layers:
  layer.trainable=False"""
print ("Model ready to be Compiled")

In [ ]:
model_pre.compile(optimizer="RMSprop", loss='categorical_crossentropy', metrics=["accuracy"])  
model_pre.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath="model2.hdf5", verbose=1, save_best_only=True, monitor="val_accuracy")

history = model.fit(train_generator, 
                    epochs=no_epochs, 
                    callbacks=[checkpointer], 
                    validation_data=val_generator, 
                    verbose = 1,
                    workers = 20)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('5.jpg')

In [ ]:
fig2 = plt.figure(figsize=(5,5))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
fig2.savefig('6.jpg')

# Baseline Model with Pretrained Weights and Freeze Layers (128 x 128)

In [ ]:
# Hyperparameters
batch_size = 100
num_classes = 200
no_epochs = 10
target_size = (128, 128)
input_shape = (128, 128, 3)

In [ ]:
# Create hash map for images and corresponding labels for Training data
train_data = []
for folder in sorted(os.listdir(train_path)):
    for file in sorted(os.listdir(train_path+folder+'/images')):
        train_data.append((folder, file))
training_df = pd.DataFrame(train_data, columns=['label', 'id'])
training_df['filename'] = train_path+training_df['label']+'/images/'+ training_df['id']

# Create hash map for images and corresponding labels for Validation data
validation_df = pd.read_csv(val_labels_path,sep="\t",header=None)
validation_df.columns=["id","label","X_min","X_max","Y_min","Y_max"]
validation_df['filename'] = val_path + validation_df['id']

image_generator = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = image_generator.flow_from_dataframe(
    dataframe=training_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",   
    shuffle=True)

val_generator = image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True)    

test_generator = image_generator.flow_from_directory(
    directory=test_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

In [ ]:
base_model = transfer_model(weights="imagenet",
                            include_top=False,
                            input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(200,activation='softmax')(x)
model_freeze = Model(base_model.input, predictions)
for layer in base_model.layers:
  layer.trainable=False
print ("Model ready to be Compiled")

In [ ]:
model_freeze.compile(optimizer="RMSprop", loss='categorical_crossentropy', metrics=["accuracy"])  
model_freeze.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath="model3.hdf5", verbose=1, save_best_only=True, monitor="val_accuracy")
history = model_freeze.fit(train_generator, 
                    epochs=no_epochs, 
                    callbacks=[checkpointer], 
                    validation_data=val_generator, 
                    verbose = 1,
                    workers = 20)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
fig.savefig('7.jpg')
plt.show()

In [ ]:
fig2 = plt.figure(figsize=(5,5))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
fig2.savefig('8.jpg')
plt.show()

# Baseline Model - Pretrained Weights, Freezed Layers and Data Augmentation(128 x 128)

In [ ]:
# Hyperparameters
batch_size = 100
num_classes = 200
no_epochs = 10
target_size = (128, 128)
input_shape = (128, 128, 3)

In [ ]:
# Create hash map for images and corresponding labels for Training data
train_data = []
for folder in sorted(os.listdir(train_path)):
    for file in sorted(os.listdir(train_path+folder+'/images')):
        train_data.append((folder, file))
training_df = pd.DataFrame(train_data, columns=['label', 'id'])
training_df['filename'] = train_path+training_df['label']+'/images/'+ training_df['id']

# Create hash map for images and corresponding labels for Validation data
validation_df = pd.read_csv(val_labels_path,sep="\t",header=None)
validation_df.columns=["id","label","X_min","X_max","Y_min","Y_max"]
validation_df['filename'] = val_path + validation_df['id']

image_generator = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = image_generator.flow_from_dataframe(
    dataframe=training_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",   
    shuffle=True)

val_generator = image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True)    

test_generator = image_generator.flow_from_directory(
    directory=test_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

data_augmentation = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomRotation(0.3),
        layers.experimental.preprocessing.Normalization()
        layers.experimental.preprocessing.RandomFlip(),
        layers.experimental.preprocessing.RandomContrast(0.3),
        layers.experimental.preprocessing.RandomTranslation(0.2, 0.2, fill_mode='wrap', interpolation='bilinear')
    ]
)

In [ ]:
inputs = Input(input_shape)
x = data_augmentation(inputs)
base_model = transfer_model(weights="imagenet",
                            include_top=False,
                            input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(200,activation='softmax')(x)
model_freeze = Model(inputs, predictions)
for layer in base_model.layers:
  layer.trainable=False
print ("Model ready to be Compiled")

In [ ]:
model_freeze.compile(optimizer="RMSprop", loss='categorical_crossentropy', metrics=["accuracy"])  
model_freeze.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath="model3.hdf5", verbose=1, save_best_only=True, monitor="val_accuracy")
history = model_freeze.fit(train_generator, 
                    epochs=no_epochs, 
                    callbacks=[checkpointer], 
                    validation_data=val_generator, 
                    verbose = 1,
                    workers = 20)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('9.jpg')

In [ ]:
fig2 = plt.figure(figsize=(5,5))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
fig2.savefig('10.jpg')

# Baseline Model (224 x 224)

In [ ]:
# Hyperparameters
batch_size = 50
num_classes = 200
no_epochs = 10
target_size = (224, 224)
input_shape = (224, 224, 3)

In [ ]:
# Create hash map for images and corresponding labels for Training data
train_data = []
for folder in sorted(os.listdir(train_path)):
    for file in sorted(os.listdir(train_path+folder+'/images')):
        train_data.append((folder, file))
training_df = pd.DataFrame(train_data, columns=['label', 'id'])
training_df['filename'] = train_path+training_df['label']+'/images/'+ training_df['id']

# Create hash map for images and corresponding labels for Validation data
validation_df = pd.read_csv(val_labels_path,sep="\t",header=None)
validation_df.columns=["id","label","X_min","X_max","Y_min","Y_max"]
validation_df['filename'] = val_path + validation_df['id']

image_generator = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = image_generator.flow_from_dataframe(
    dataframe=training_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",   
    shuffle=True)

val_generator = image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=None,
    x_col="filename",
    y_col="label",
    target_size =target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True)    

test_generator = image_generator.flow_from_directory(
    directory=test_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

In [ ]:
base_model = transfer_model(weights=None,
                            include_top=False,
                            input_shape = input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(200,activation='softmax')(x)
model = Model(base_model.input, predictions)

In [ ]:
model.compile(optimizer="RMSprop", loss='categorical_crossentropy', metrics=["accuracy"])  
model.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath="model_base224.hdf5", verbose=1, save_best_only=True, monitor="val_accuracy")

In [ ]:
history2 = model.fit(train_generator, 
                    epochs=no_epochs, 
                    callbacks=[checkpointer], 
                    validation_data=val_generator, 
                    verbose = 1,
                    workers = 20)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('11.jpg')

In [ ]:
fig2 = plt.figure(figsize=(5,5))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.show()
fig2.savefig('12.jpg')